In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob

In [2]:
model = models.resnet50(pretrained=True)

layer = model._modules.get('avgpool')

model.eval()

scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name):
    img = Image.open(image_name)
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    my_embedding = torch.zeros(1,2048,1,1)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    h = layer.register_forward_hook(copy_data)
    model(t_img)
    h.remove()
    return my_embedding.numpy()

/home/sayan/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [4]:
data_dir = '/home/sayan/Desktop/code_2023/RS_data/PatternNet'

In [ ]:
# list of image files and corresponding labels
files_list = []
label_list = []
x = os.walk(data_dir, topdown=False)
j=-1
for path,d,filelist in sorted(x):
    
    for filename in sorted(filelist):
        file_glob = os.path.join(path, filename)
        files_list.extend(glob.glob(file_glob))
        label_list.append(j)

    j=j+1


In [ ]:
data=np.zeros([len(files_list), 2049])
print(len(files_list))

In [ ]:
# feature generation
for i in range(len(files_list)):
    vec=get_vector(files_list[i])
    vec=vec.flatten()
    data[i,:-1]=vec
    data[i,-1]=label_list[i]
    print(i)

In [ ]:
c = np.savetxt('/home/user/Desktop/desk/dataset/image/rs_image/im256*256/PatternNet_closed.csv', data, delimiter =', ') 

In [ ]:
sw=np.loadtxt(open('/home/user/Desktop/desk/dataset/image/rs_image/im256*256/PatternNet_closed.csv', "rb"), delimiter=",")